## Calculating gradients in three ways

Using the example from Chris Olah's [Calculus on Computational Graphs: Backpropagation](http://colah.github.io/posts/2015-08-Backprop/), we'll show how to calcuate gradients for the function `f(a, b) = (a + b) * (b + 1)` in three different ways:

1. Numerically (nudge each parameter a little) -- simple, fun, slow.
2. Analytically (backprop by hand, remembering our rules from calculus) -- efficient, tricky.
3. Analytically (reverse mode autodiff in TensorFlow 2.0) -- efficient, automatic.

After that, as a recommended exercise, I suggest watching this helpful [video](https://www.youtube.com/watch?v=d14TUNcbn1k) from Stanford's cs231 -- one of the clearest explanations of backprop on the web. Try to calculate the gradients for one of their examples using these methods, and verify you can get the same result with each. 

* I've included a solution for the first function: `f(x, y, z) = (x + y) * z`. 

At the end, there's a matrix example for a dense layer with ReLU activation.

* f(W, x) = ReLU(Wx)

In [0]:
!pip install tf-nightly-2.0-preview

In [0]:
import numpy as np
import tensorflow as tf

print("You have version", tf.__version__)
assert tf.__version__ >= "2.0" # TensorFlow ≥ 2.0 required

You have version 2.0.0-dev20190206


## First example

Our first function will be `f(a, b) = (a + b) * (b + 1)`. 

### Forward pass

In [0]:
# forward pass, creating intermediate variables as we go
def forward(a, b):
  c = a + b
  d = b + 1
  f = c * d
  return f

In [0]:
forward(2,1)

6

### Numeric gradient

In [0]:
def numeric_gradient(my_function, params, h=1e-4):
  
  # Vector of partial derivatives
  grads = []
  
  # Nudge each parameter by h
  # calculate how much the function changes
  for i in range(len(params)):
    
    orginal_val = params[i]
    
    # f(x + h)
    params[i] = orginal_val + h
    plus_h = my_function(*params) 
        
    # f(x - h)
    params[i] = orginal_val - h
    minus_h = my_function(*params)
        
    # Partial derivative
    grad = (plus_h - minus_h) / (2 * h)
    grads.append(grad)
    
    # reset
    params[i] = orginal_val
       
  return grads

In [0]:
da, db = numeric_gradient(my_function=forward, params=[2, 1])
print ("Numeric gradient. da %0.2f, db %0.2f" % (da, db))

Numeric gradient. da 2.00, db 5.00


### Analytic gradient (by hand)
Backprop by hand, if we rememeber our calculus rules.

In [0]:
def backprop_by_hand(a, b):
  
  # forward pass, keeping track of intermediate variables
  c = a + b
  d = b + 1
  f = c * d
  
  df = 1 # Gradient of f itself is 1 (this would be replaced by loss in a model)
  dc = d # Product rule
  dd = c # Product rule

  dc_da = 1 # Sum rule
  da = dc * dc_da
  
  dc_db = 1 # Sum rule
  db_path_1 = dc * dc_db
  
  dd_db = 1 # Sum rule
  db_path_2 = dd * dd_db
  
  db = db_path_1 + db_path_2 # Sum over paths (multivariate chain rule)
  
  return da, db

da, db = backprop_by_hand(a=2, b=1)
print ("Gradient by backprop. da %0.2f, db %0.2f" % (da, db))

Gradient by backprop. da 2.00, db 5.00


### Analytic gradient (autodiff)
Reverse mode autodiff using TensorFlow.

Using a GradientTape ([doc](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/GradientTape), [short tutorial](https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/eager/automatic_differentiation.ipynb)).

In [0]:
params = [tf.Variable(2.0), tf.Variable(1.0)]

with tf.GradientTape() as tape:
  result = forward(*params)
grads = tape.gradient(result, params)

print("Analytic gradient (by autodiff). da %0.2f, db %0.2f" % 
      (grads[0].numpy(), grads[1].numpy()))

Analytic gradient (by autodiff). da 2.00, db 5.00


## Second example

Here's `f(x, y, z) = (x + y) * z` in the same three ways.

In [0]:
def forward(x, y, z):
  return (x + y) * z

In [0]:
forward(x=1, y=2, z=-4)

-12

### Numeric gradient
Same method as above.

In [0]:
dx, dy, dz = numeric_gradient(my_function=forward, params=[1, 2, -4])
print ("Numeric gradient. dx %0.2f, dy %0.2f, dz %0.2f" % (dx, dy, dz))

Numeric gradient. dx -4.00, dy -4.00, dz 3.00


### Analytic gradient (by hand)
Backprop by hand, if we rememeber our calculus rules.

In [0]:
def backprop_by_hand(x, y, z):
  
  # Calculate the forward pass, creating intermediate variables as we go.
  q = x + y
  f = q * z
  
  # Propagate the gradient on the function's output 
  # backward through intermediate variables, until we reach the inputs.
  
  # Gradient on the function is just itself. In a model, this would 
  # be replaced by the loss.
  df = 1 
  
  # By product rule.
  dq = z
  
  # By product rule.
  dz = q
  
  dq_dx = 1 # By sum rule
  dq_dy = 1 # By sum rule
  
  # Backprop gradient on q into x
  dx = dq * dq_dx
  
  # Backprop gradient on q into y
  dy = dq * dq_dy
    
  return dx, dy, dz

In [0]:
dx, dy, dz = backprop_by_hand(1., 2., -4.)
print ("Gradient by backprop. dx %0.2f, dy %0.2f, dz %0.2f" % (dx, dy, dz))

Gradient by backprop. dx -4.00, dy -4.00, dz 3.00


### Analytic gradient (autodiff)
Reverse mode autodiff using TensorFlow.

In [0]:
params = [tf.Variable(1.0), tf.Variable(2.0), tf.Variable(-4.0)]

with tf.GradientTape() as tape:
  result = forward(*params)
grads = tape.gradient(result, params)

print("Analytic gradient (by autodiff). dx %0.2f, dy %0.2f dz %0.2f" % 
      (grads[0].numpy(), grads[1].numpy(), grads[2].numpy()))

Analytic gradient (by autodiff). dx -4.00, dy -4.00 dz 3.00


## Third example

`f(W, x) = ReLU(Wx)`

In [0]:
def forward(W, x):
    wx = tf.matmul(W,x)
    result = tf.nn.relu(wx)
    return result

In [0]:
W = np.array([[-0.5, 0.2],
              [-0.3, 0.8]])                  
x = np.array([0.2, 0.4])  
x = np.expand_dims(x,1)   
forward(W,x)

<tf.Tensor: id=194, shape=(2, 1), dtype=float64, numpy=
array([[0.  ],
       [0.26]])>

### Numeric gradient

In [0]:
# Updated code that supports matmul
def numeric_gradient_v2(my_function, weights, inputs, h=1e-4):
  
  # Vectors of partial derivatives
  grads = np.zeros_like(weights)
  
  for i in range(grads.shape[0]):
    for j in range(grads.shape[1]):
      
      original = weights[i,j]
      
      weights[i,j] = original + h
      plus_h = my_function(weights, inputs)
      
      weights[i,j] = original - h
      minus_h = my_function(weights, inputs)
      
      # partial
      grads[i,j] = np.sum(plus_h - minus_h) / (2 * h)
      
      # reset
      weights[i,j] = original
      
  return grads

In [0]:
dw = numeric_gradient_v2(forward, weights=W, inputs=x)
print ("Numeric gradient. dw\n" % dw)

Numeric gradient. dw



### Analytic gradient (by hand)
Backprop by hand, if we rememeber our calculus rules.

In [0]:
# Backprop
def backward(W, x):
  q = np.maximum(0, np.matmul(W, x))
  dq = np.ones_like(q)
  dq[q <= 0] = 0  
  dw = np.matmul(dq, x.T)
  return dw

dw = backward(W,x)
print ("Gradient by hand\n", dw)

Gradient by hand
 [[0.  0. ]
 [0.2 0.4]]


### Analytic gradient (autodiff)
Reverse mode autodiff using TensorFlow.

In [0]:
W = tf.Variable([[-0.5, 0.2],
                 [-0.3, 0.8]])                  
x = tf.Variable([0.2, 0.4])  
x = tf.expand_dims(x,1)   

params = [W, x]
with tf.GradientTape() as tape:
  result = forward(*params)
grads = tape.gradient(result, W)

print("Analytic gradient (by autodiff). dw", grads)

Analytic gradient (by autodiff). dw tf.Tensor(
[[0.  0. ]
 [0.2 0.4]], shape=(2, 2), dtype=float32)
